In [2]:
import pandas as pd

data_exp = pd.read_csv('/Users/camille/repo/Hetic/projet_gouv/scraping/export-experiences.csv', sep='\t', encoding='latin-1')


/var/folders/vh/yn_93j315gn6jdvz7f2d73_m0000gn/T/ipykernel_16475/255472605.py:3: DtypeWarning: Columns (31,50) have mixed types. Specify dtype option on import or set low_memory=False.
  data_exp = pd.read_csv('/Users/camille/repo/Hetic/projet_gouv/scraping/export-experiences.csv', sep='\t', encoding='latin-1')


In [3]:



new_df = pd.DataFrame()

# Colonne 'Information/Explication_neg'
new_df['pos_info'] = data_exp['Information/Explication'].apply(lambda x: 1 if x == 'Positif' else 0)
new_df['neg_info'] = data_exp['Information/Explication'].apply(lambda x: 1 if x == 'Négatif' else 0)

new_df['pos_access'] = data_exp['Accessibilité'].apply(lambda x: 1 if x == 'Positif' else 0)
new_df['neg_access'] = data_exp['Accessibilité'].apply(lambda x: 1 if x == 'Négatif' else 0)

new_df['pos_relation'] = data_exp['Relation'].apply(lambda x: 1 if x == 'Positif' else 0)
new_df['neg_relation'] = data_exp['Relation'].apply(lambda x: 1 if x == 'Négatif' else 0)

new_df['pos_reactivite'] = data_exp['Réactivité'].apply(lambda x: 1 if x == 'Positif' else 0)
new_df['neg_reactivite'] = data_exp['Réactivité'].apply(lambda x: 1 if x == 'Négatif' else 0)

new_df['pos_simplicite'] = data_exp['Simplicité/Complexité'].apply(lambda x: 1 if x == 'Positif' else 0)
new_df['neg_simplicite'] = data_exp['Simplicité/Complexité'].apply(lambda x: 1 if x == 'Négatif' else 0)

# Colonne 'text'
new_df['Description'] = data_exp['Description']


In [4]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(new_df, random_state=42, test_size=0.30, shuffle=True)

In [5]:
train_text = train['Description']
test_text = test['Description']

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
vectorizer.fit(train_text)
vectorizer.fit(test_text)

TfidfVectorizer(ngram_range=(1, 3), strip_accents='unicode')

In [7]:
categories = list(new_df.columns.values)
categories = categories[:-1]
print(categories)

['pos_info', 'neg_info', 'pos_access', 'neg_access', 'pos_relation', 'neg_relation', 'pos_reactivite', 'neg_reactivite', 'pos_simplicite', 'neg_simplicite']


In [16]:
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier

In [14]:
x_train = vectorizer.transform(train_text)
y_train = train.drop(labels = ['Description'], axis=1)

x_test = vectorizer.transform(test_text)
y_test = test.drop(labels = ['Description'], axis=1)

In [11]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
#printmd('**bold**')

In [17]:
%%time

# Using pipeline for applying logistic regression and one vs rest classifier
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])

for category in categories:
    printmd('**Processing {} comments...**'.format(category))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(x_train, train[category])
    
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(x_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))
    print("\n")

**Processing pos_info comments...**

Test accuracy is 0.9076971708550656




**Processing neg_info comments...**

Test accuracy is 0.7314683104156788




**Processing pos_access comments...**

...

Test accuracy is 0.80891417733523




**Processing pos_simplicite comments...**

Test accuracy is 0.9089615931721194




**Processing neg_simplicite comments...**

Test accuracy is 0.6609767662399242


CPU times: user 116 ms, sys: 181 ms, total: 296 ms
Wall time: 7.87 s
